In [1]:
from collections import defaultdict

with open("inputs/day11-input.txt") as f:
    puzzle = list(map(int, f.read().split(",")))
    
intcode = defaultdict(int)
for i, x in enumerate(puzzle):
    intcode[i] = x

## Part 1

In [2]:
%%time

from typing import Dict, List

class IntcodeComputer:
    def __init__(self, intcode: Dict[int, int]):
        self.intcode = intcode
        self.input = []

        
    def get_param_index(self, instruction_index: int, parameter: int) -> int:
        instruction = f"{self.intcode[instruction_index]:05d}"
        mode = instruction[-2 - parameter]
        is_position  = (mode == "0")
        is_immediate = (mode == "1")
        is_relative  = (mode == "2")
        
        param_index = instruction_index + parameter
        param_value = self.intcode[param_index]
        
        if is_position:
            return param_value
        
        elif is_immediate:
            return param_index
        
        elif is_relative:
            return self.relative_base + param_value
        
    def get_param(self, instruction_index: int, parameter: int) -> int:
        return self.intcode[self.get_param_index(instruction_index, parameter)]


    def get_opcode(self, instruction_index: int) -> int:
        opcode = str(self.intcode[instruction_index])[-2:]
        return int(opcode)


    def run_program(self, _input: List[int]=[]) -> bool:
        index = 0
        self.relative_base = 0
        
        while True:
            opcode = self.get_opcode(index)

            if opcode == 99:
                yield True
                break

            if opcode == 1:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 + in2
                index += 4

            elif opcode == 2:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 * in2
                index += 4

            elif opcode == 3:
                in1 = self.get_param_index(index, 1)

                self.intcode[in1] = self.input.pop(0)
                index += 2

            elif opcode == 4:
                in1 = self.get_param(index, 1)

                self.result = in1
                index += 2
                yield False

            elif opcode == 5:
                in1 = self.get_param(index, 1)
                if in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 6:
                in1 = self.get_param(index, 1)
                if not in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 7:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 < in2)
                index += 4

            elif opcode == 8:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 == in2)
                index += 4
                
            elif opcode == 9:
                in1 = self.get_param(index, 1)
                
                self.relative_base += in1
                index += 2
    

computer = IntcodeComputer(intcode.copy())
hull = defaultdict(int)

directions = {
    0: (0, -1),
    1: (1, 0),
    2: (0, 1),
    3: (-1, 0)
}
direction = 0
position = (0, 0)

def move(position, movement):
    return (position[0] + movement[0], position[1] + movement[1])

program = computer.run_program()

while True:
    computer.input.append(hull[position])
    
    halted = next(program)
    
    if halted:
        break
        
    hull[position] = computer.result
    next(program)
    turn = 1 if computer.result else -1
    direction = (direction + turn) % 4
    position = move(position, directions[direction])
    
print(len(hull.keys()))

2238
CPU times: user 297 ms, sys: 0 ns, total: 297 ms
Wall time: 299 ms


In [3]:
%%time

computer = IntcodeComputer(intcode.copy())
hull = defaultdict(int)
direction = 0
position = (0, 0)
hull[position] = 1
program = computer.run_program()

while True:
    computer.input.append(hull[position])
    
    halted = next(program)
    
    if halted:
        break
        
    hull[position] = computer.result
    next(program)
    turn = 1 if computer.result else -1
    direction = (direction + turn) % 4
    position = move(position, directions[direction])
    
min_x = min(hull.keys(), key=lambda x: x[0])[0]
min_y = min(hull.keys(), key=lambda x: x[1])[1]
max_x = max(hull.keys(), key=lambda x: x[0])[0]
max_y = max(hull.keys(), key=lambda x: x[1])[1]

for y in range(min_y, max_y + 1):
    for x in range(min_x, max_x + 1):
        char = "X" if hull[(x, y)] else " "
        print(char, end="")
    print()

 XXX  X  X XXXX XXX   XX  XXXX XXX  XXX    
 X  X X X  X    X  X X  X    X X  X X  X   
 X  X XX   XXX  X  X X  X   X  X  X X  X   
 XXX  X X  X    XXX  XXXX  X   XXX  XXX    
 X    X X  X    X    X  X X    X X  X      
 X    X  X X    X    X  X XXXX X  X X      
CPU times: user 57.6 ms, sys: 9 ms, total: 66.6 ms
Wall time: 63.4 ms
